In [ ]:
import requests
import json
import uuid

In [ ]:
# 是否只选每种类别的第一个（加速）
flag = True

# 选择座位
libName = "东馆2层"
typeName = "交流空间（二层东）"
roomName = "二层④区交流空间1排01组"
roomId = "bca19acc7ae1497089ceee875925587e"

# 映射后账户密码
account = "4521D610DC9460D22004EC76E32BF7E1"
upass = "6BED22231B81701E9F89F37B269CC290"

In [ ]:
# 选择时间
# 8点：480，9点：540，10点：600，11点：660，12点：720，13点：780，14点：840，15点：900，16点：960，17点：1020，18点：1080，19点：1140，20点：1200，21点：1260，22点：1320
# 13点
time1 = "780"
# 17点
time2 = "1020"
# 21点
time3 = "1260"

In [ ]:
# 生成UUID（非必须）
ticketCode = str(uuid.uuid4()).replace('-','')
print("ticketCode:",ticketCode)

In [ ]:
# 登录，获取基本信息
headers = {
    "Host":"libwx.cau.edu.cn",
    "User-Agent":f"Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1 Edg/119.0.0.0",
    "Referer":"http://libwx.cau.edu.cn/remote/static/phoneloginHandler"
}
params = {
    'type':'discuss',
    'openid':'',
    'account':account,
    'upass':upass,
    'ticketCode':ticketCode,
    'nickname':'',
    'isFlag':'',
    'headimgurl':'',
    'sign':''
}
response = requests.get("http://libwx.cau.edu.cn/space/static/dowechatlogin?type=discuss", headers=headers, params=params)
cookie = response.request.headers['Cookie']
libId = response.text[response.text.find(libName)-34:response.text.find(libName)-2]
typeId = response.text[response.text.find(typeName)-78:response.text.find(typeName)-46]
csrf = response.text[response.text.find("<meta name=\"_csrf\" content=")+28:response.text.find("<meta name=\"_csrf\" content=")+64]
print('cookie:',cookie)
print('libId:',libId)
print('typeId:',typeId)
print('csrf',csrf)

In [ ]:
# 获取时间
headers = {
    "Host":"libwx.cau.edu.cn", 
    "User-Agent":f"Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1 Edg/119.0.0.0",
    "Referer":"http://libwx.cau.edu.cn/space/discuss/mobileIndex?type=discuss",
    "X-Requested-With":"XMLHttpRequest",
    "Cookie":cookie
}
url = "http://libwx.cau.edu.cn/space/discuss/getSelectDays/" + typeId
response = requests.get(url, headers=headers)
today = json.loads(response.text)["params"]["days"][0]["day"]
print('today:', today)


In [ ]:
# 获取座位
headers = {
    "Host":"libwx.cau.edu.cn",
    "Origin":"http://libwx.cau.edu.cn",
    "Content-Length":"26",
    "Content-Type":"application/x-www-form-urlencoded; charset=UTF-8",
    "User-Agent":f"Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1 Edg/119.0.0.0",
    "Referer":"http://libwx.cau.edu.cn/space/discuss/mobileIndex?type=discuss",
    "X-Requested-With":"XMLHttpRequest",
    "X-Csrf-Token":csrf,
    "Cookie":cookie
}
size = "1" if flag else "100"
payload = {
    "currentPage":"1",
    "pageSize":size
}
url = "http://libwx.cau.edu.cn/space/discuss/findRoom/" + typeId + "/" + libId + "/" + today
response = requests.post(url, headers=headers, data=payload)
dict = json.loads(response.text)
ruleId = dict["params"]["ruleId"]
print("ruleId:",ruleId)
print('roomId:', roomId)
# for item in dict["params"]["rooms"]["pageList"]:
#     if item["rname"] == roomName:
#         roomId = item["id"]
#         print("roomId:",roomId)
#         break

# 获取UUID
headers = {
    "Host":"libwx.cau.edu.cn", 
    "User-Agent":f"Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1 Edg/119.0.0.0",
    "Cookie":cookie
}
params = {
    "roomid":roomId,
    "ustime":"1260",
    "uetime":"1320",
    "selectDate":today,
    "ruleId":ruleId,
    "mobile":"true",
    "linkSign":"discuss"
}
response = requests.get("http://libwx.cau.edu.cn/space/discuss/openAppointDetail", headers=headers, params=params)
uuid_ = response.text[response.text.find("UUID")+7:response.text.find("UUID")+15]
print("uuid:",uuid_)

In [ ]:
# today = '2024-03-21'
print(today)

In [ ]:
# 预定（可修改today）
headers = {
    "Host":"libwx.cau.edu.cn",
    "Origin":"http://libwx.cau.edu.cn",
    "Content-Length":"261",
    "Content-Type":"application/json",
    "User-Agent":f"Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1 Edg/119.0.0.0",
    "X-Requested-With":"XMLHttpRequest",
    "X-Csrf-Token":csrf,
    "Cookie":cookie
}
payload = {
    "LOCK":"true",
    "UUID":uuid_,
    "room_exp":"[]",
    "ruleId":ruleId,
    "usercount":"2",
    "users":"2020308250121 2020308130408",
    "_currentday":today,
    "_stime":1260,
    "_etime":1320,
    "_roomid":roomId,
    "_seatno":"0"
}
response = requests.post("http://libwx.cau.edu.cn/space/form/dynamic/saveFormLock", headers=headers, json=payload)
print("预定1：", json.loads(response.text))